This file has the code to obtain syllable transition values from Onset-Offset data, and process the file

Date: 20/9/2022

In [42]:
import math
import pandas
import os
import numpy as np

In [43]:
#set directory
os.chdir('D:\\4th Year\\Semester 7\\BI4313 Sem Project\\IN-comparative-analysis\\IN-comparative-analysis\\OnsetOffsetFiles')

In [44]:
#Import text file as csv, choose header false and give column names manually, because they don't seem to align
#Use 'print(data.columns)' to get the list of column names
data = pandas.read_csv('M09.txt', sep = '\t', header=None, names = ['FileName', 'Syll #', 'Syll Label',
       ' Syll Onset (ms)', ' Syll Offset (ms)', ' Syll Duration (sec)', ' Mean Frequency (Hz)',
       ' Entropy', 'Log Amplitude (dB)', 'Pitch Goodness', 'FrequencyModulation',
       'AmplitudeModulation', 'EntropyVariance'])

#Drop the first row because column names are repeated there
data = data.iloc[1:, :]

#Convert onset, offset and syllable duration to float type
data[' Syll Onset (ms)'] = data[' Syll Onset (ms)'].astype(float)
data[' Syll Offset (ms)'] = data[' Syll Offset (ms)'].astype(float)
data[' Syll Duration (sec)'] = data[' Syll Duration (sec)'].astype(float)
#data[' Syll Offset (ms)']

In [45]:
data

,FileName,Syll #,Syll Label,Syll Onset (ms),Syll Offset (ms),Syll Duration (sec),Mean Frequency (Hz),Entropy,Log Amplitude (dB),Pitch Goodness,FrequencyModulation,AmplitudeModulation,EntropyVariance
1,20150622_M09_paired_01.wav,1,i,9561.84,9579.75,0.017914,2537.39,-3.5513,11.4333,45.3959,45.3802,-0.0129652,0.455702
2,20150622_M09_paired_01.wav,2,a,9734.56,9977.57,0.243016,3284.31,-2.9849,19.2989,41.5009,42.234,-0.00630934,0.0866028
3,20150622_M09_paired_01.wav,3,b,10056.80,10129.20,0.072358,2742.15,-4.25822,11.7839,37.0944,8.50343,-0.0063626,0.340397
4,20150622_M09_paired_01.wav,4,c,10145.60,10195.60,0.050091,4611.29,-2.52174,22.2206,43.7937,45.7616,-0.06946,0.147445
5,20150622_M09_paired_01.wav,5,d,10247.60,10379.90,0.132222,4322.4,-5.17878,28.4537,24.845,11.7422,-0.0117026,0.576616
...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,20150622_M09_paired_08.wav,93,m,59257.70,59344.30,0.086644,6907.44,-5.37949,29.8242,14.8688,24.0078,-0.00699179,1.70616
544,20150622_M09_paired_08.wav,94,m,59369.30,59455.20,0.085896,6901.92,-5.4639,29.5717,13.9225,21.1852,-0.00746598,1.79158
545,20150622_M09_paired_08.wav,95,m,59482.20,59570.80,0.088549,6916.15,-5.41635,27.2352,13.2342,18.6592,-0.0103777,2.12493
546,20150622_M09_paired_08.wav,96,m,59609.00,59691.30,0.082331,6679.02,-5.36,31.9068,19.1618,17.0812,-0.0275406,2.82375


In [88]:


#Problem - this code is not updating index number as it adds a 'start', so from the 2nd break onwards,
#the 'start' label is misplaced by one row. To correct this, I'll create a list of indices and insert
#the 'start' row using this list

#t_offset = data[' Syll Offset (ms)']
#new_row = [0, 0, 'Start', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

#for i in range(1, len(data)-1):
#    diff = t_offset[i+1] - t_offset[i]
#    if diff > 2000:
#        data = pandas.DataFrame(np.insert(data.values, i, new_row, axis=0))


In [46]:
#To add a 'start' in syllable label column after end of every bout

t_offset = data[' Syll Offset (ms)']
new_row = [0, 0, 'Start', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
index_list = []

for i in range(1, len(data)-1):
    diff = t_offset[i+1] - t_offset[i]
    if diff > 2000:
        index_list.append(i)

#So now we have a list of indices where 'start' row should be inserted
#Adjust the index to correct for where the row will be inserted
for i in range(1, len(index_list)):
    index_list[i] += 1*i

#Insert the new_row at these indices
#PS. If this doesn't make sense, remove the correction, do a simple for loop and see what happens
for i in index_list:
    data = pandas.DataFrame(np.insert(data.values, i, new_row, axis=0))

#To confirm
#print(data)
#data.to_csv('M09_tp.csv', header = False, index = False)




In [47]:
data.columns = ['FileName', 'Syll #', 'Syll Label',
       ' Syll Onset (ms)', ' Syll Offset (ms)', ' Syll Duration (sec)', ' Mean Frequency (Hz)',
       ' Entropy', 'Log Amplitude (dB)', 'Pitch Goodness', 'FrequencyModulation',
       'AmplitudeModulation', 'EntropyVariance']

data

,FileName,Syll #,Syll Label,Syll Onset (ms),Syll Offset (ms),Syll Duration (sec),Mean Frequency (Hz),Entropy,Log Amplitude (dB),Pitch Goodness,FrequencyModulation,AmplitudeModulation,EntropyVariance
0,20150622_M09_paired_01.wav,1,i,9561.84,9579.75,0.017914,2537.39,-3.5513,11.4333,45.3959,45.3802,-0.0129652,0.455702
1,20150622_M09_paired_01.wav,2,a,9734.56,9977.57,0.243016,3284.31,-2.9849,19.2989,41.5009,42.234,-0.00630934,0.0866028
2,20150622_M09_paired_01.wav,3,b,10056.8,10129.2,0.072358,2742.15,-4.25822,11.7839,37.0944,8.50343,-0.0063626,0.340397
3,20150622_M09_paired_01.wav,4,c,10145.6,10195.6,0.050091,4611.29,-2.52174,22.2206,43.7937,45.7616,-0.06946,0.147445
4,20150622_M09_paired_01.wav,5,d,10247.6,10379.9,0.132222,4322.4,-5.17878,28.4537,24.845,11.7422,-0.0117026,0.576616
...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,20150622_M09_paired_08.wav,93,m,59257.7,59344.3,0.086644,6907.44,-5.37949,29.8242,14.8688,24.0078,-0.00699179,1.70616
584,20150622_M09_paired_08.wav,94,m,59369.3,59455.2,0.085896,6901.92,-5.4639,29.5717,13.9225,21.1852,-0.00746598,1.79158
585,20150622_M09_paired_08.wav,95,m,59482.2,59570.8,0.088549,6916.15,-5.41635,27.2352,13.2342,18.6592,-0.0103777,2.12493
586,20150622_M09_paired_08.wav,96,m,59609.0,59691.3,0.082331,6679.02,-5.36,31.9068,19.1618,17.0812,-0.0275406,2.82375


In [48]:
#To get the occurrence of number of syllables
#And confirm that this is the same as row_sum of trans_matrix

syl_n = data['Syll Label'].value_counts()

#syl_n

m        89
p        83
i        51
q        51
Start    41
h        33
b        26
e        24
d        24
c        24
g        23
f        22
a        22
l        22
n        18
r        15
s        11
k         7
D         1
x         1
Name: Syll Label, dtype: int64

In [49]:
syl = data['Syll Label']
#print(syl)

#Get the set of unique syllables for the individual with their own index (so convert array to list)
uniq_syl = syl.unique()
uniq_syl = uniq_syl.tolist()
#print(uniq_syl)

#Get the number of syllables to create a N x N matrix for transition
N = len(uniq_syl)
trans_matrix = np.array(np.zeros((N, N), dtype = int))

#uniq_syl.index('a')


0      i
1      a
2      b
3      c
4      d
      ..
583    m
584    m
585    m
586    m
587    m
Name: Syll Label, Length: 588, dtype: object


1

In [50]:

#Convert 'object' type to array to get indices
syl = np.array(syl)

#This gives the number of syllable transitions
for i in range(len(syl)-1):
    a = uniq_syl.index(syl[i])
    b = uniq_syl.index(syl[i+1])
    trans_matrix[a, b] += 1

#trans_matrix

In [51]:
#Divide the number of transitions by total number of occurrence of that syllable to get transition probability
trans_matrix = (trans_matrix.T/trans_matrix.sum(axis=1)).T
trans_matrix = np.around(trans_matrix, 2)

#print(np.sum(trans_matrix[0,]))
#print(trans_matrix)

In [52]:
#Add the labels of syllables as an extra row and column
syl_name = np.array(uniq_syl)

trans_prob = np.concatenate([[syl_name], trans_matrix])
#trans_prob

In [53]:
#Add a '0' in the beginning to insert this as a column
syl_name = np.concatenate([[0], syl_name])

trans_prob = np.insert(trans_prob, 0, syl_name, axis = 1)
#trans_prob

#Convert to dataframe to save as .csv file
transition_probability = pandas.DataFrame(trans_prob)
transition_probability.to_csv('M09_trans_prob.csv', header = False, index = False)

#There are values lesser than 0.05 that can be removed through an IF loop perhaps
#But we can keep the values and code the diagram to not show them
#This file contains the low-occuring syllables as well